## aihub의 뉴스톡해 데이터
- https://aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=577 - 뉴스기사 기계독해 데이터
- 017.뉴스기사 기계독해 데이터 -> 01.데이터 -> 1.Training -> 라벨링 데이터 -> TL_text_entailment.zip

### Library

In [1]:
import os
import pandas as pd
import numpy as np
import json

### Data Load

In [ ]:
input_data_path = os.path.join(os.getcwd(), "base_data") 
output_data_path = os.path.join(os.getcwd(), "data") # 저장 경로
ox_path = os.path.join(input_data_path, "qa", "aihub", "TL_text_entailment.json")

In [ ]:
record_path = ['data', 'paragraphs', 'qas', 'answers'] # 경로상의 key들
file = json.loads(open(ox_path).read()) # json파일 열기

In [ ]:
def aihub_ox_train(input_file_path, record_path = ['data', 'paragraphs','qas','answers'], verbose = 1):
    """
    input_file_path: json파일 경로
    record_path: json파일의 접근 경로상의 key값들 ['data','paragraphs','qas','answers']
    verbose: 0 to suppress it default is 1
    """
    if verbose:
        print("Reading the json file")    
    file = json.loads(open(input_file_path).read()) # json 파일 열기
    if verbose:
        print("processing...")

    js = pd.io.json.json_normalize(file, record_path[:-1]) # record_path가 qas까지 접근, json파일을 데이터프레임으로 변환
    m = pd.io.json.json_normalize(file, record_path[:-2]) # record_path가 paragraphs까지 접근
    r = pd.io.json.json_normalize(file,record_path[:-3]) # data - 분류 카테고리 확인을 위해
    
    idx = np.repeat(r['doc_class.code'].values, m.qas.str.len()) # 개수 파악 - idx
    ndx  = np.repeat(m['context'].values,m.qas.str.len()) # 내용만 가져오기
    jsdx = js.drop(["is_impossible", "answers.clue_start", "answers.clue_text", "answers.options"], axis=1) # 불필요 열 제거
    jsdx['context'] = ndx

    main = jsdx.set_index('question_id').reset_index() # 인덱스 초기화
    main["type"] = idx
    main['c_id'] = main['context'].factorize()[0]
    if verbose:
        print("shape of the dataframe is {}".format(main.shape)) # 데이터 크기 확인
        print("Done")
    return main

In [ ]:
ox_train = aihub_ox_train(ox_path)
ox_train = ox_train[ox_train["type"]=="경제"] # 필요한 부분만 가져오기
aihub_ox = ox_train[["question", "context", "answers.text"]].reset_index(drop=True) # 인덱스 재정렬
aihub_ox.columns = ["question", "context", "answer"] # column 이름 변경
aihub_ox["answer"] = aihub_ox["answer"].replace({"Yes":0, "No":1}) # 답변 값 변경
aihub_ox # 확인

### 저장

In [ ]:
aihub_ox.to_csv(os.path.join(output_data_path, "aihub_ox.csv"), encoding="utf-8", index=False)